# Car Make and Model Recognizer
I shall explore here my tries to reach a good optimizer

## Importing Libraries

In [1]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation, PReLU, LeakyReLU
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from glob import glob
from keras import regularizers
from keras.layers.advanced_activations import PReLU
from keras import models
from keras.applications import DenseNet121

Using TensorFlow backend.


## Pre-Processing

### Data Augmentation

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.25,
                                   rotation_range = 50,
                                   horizontal_flip=True,
                                  validation_split = 0.3)
#test_datagen = ImageDataGenerator(rescale=1./255,
 #                                horizontal_flip = True)

train_data = train_datagen.flow_from_directory('My_Cars/trainCropped',
                                              target_size=(256, 256),
                                              batch_size=32,
                                              class_mode='categorical',
                                              subset='training')
test_data = train_datagen.flow_from_directory('My_Cars/trainCropped',
                                              target_size=(256, 256),
                                              batch_size=32,
                                              class_mode='categorical',
                                            subset='validation')

Found 5796 images belonging to 196 classes.
Found 2348 images belonging to 196 classes.


## Model

In [3]:
model = Sequential()


conv_base = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
model.add(conv_base)
model.add(GlobalAveragePooling2D())

model.add(Dense(196, activation='softmax'))
for layer in conv_base.layers:
  layer.trainable = False


model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


Instructions for updating:
Colocations handled automatically by placer.
29089792/29084464 [==============================] - 18s 1us/step


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 8, 8, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 196)               200900    
Total params: 7,238,404
Trainable params: 200,900
Non-trainable params: 7,037,504
_________________________________________________________________


### Train

In [5]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 300

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='Saved_Models/weights.best.from_scratch43.hdf5', 
                               verbose=1, save_best_only=True)

Saving = model.fit_generator(train_data,
          steps_per_epoch=182,  
          epochs=epochs,
          verbose=1, 
          validation_data=test_data,
          validation_steps=74,
          callbacks=[checkpointer])
# model.fit(train_data, test_data,
#                   callbacks = [checkpointer],
#                   validation_split = 0.33,
#                   epochs=epochs,
#           batch_size = 20,
#           verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
182/182 [==============================] - 148s 812ms/step - loss: 5.3641 - acc: 0.0223 - val_loss: 4.9513 - val_acc: 0.0285

Epoch 00001: val_loss improved from inf to 4.95130, saving model to Saved_Models/weights.best.from_scratch43.hdf5
Epoch 2/300
182/182 [==============================] - 137s 752ms/step - loss: 4.5797 - acc: 0.0874 - val_loss: 4.4118 - val_acc: 0.1005

Epoch 00002: val_loss improved from 4.95130 to 4.41179, saving model to Saved_Models/weights.best.from_scratch43.hdf5
Epoch 3/300
182/182 [==============================] - 133s 733ms/step - loss: 4.0374 - acc: 0.1607 - val_loss: 4.0955 - val_acc: 0.1410

Epoch 00003: val_loss improved from 4.41179 to 4.09553, saving model to Saved_Models/weights.best.from_scratch43.hdf5
Epoch 4/300
182/182 [==============================] - 133s 732ms/step - loss: 3.6423 - acc: 0.2287 - val_loss: 3.7412 - val_acc: 0.2266

Epoch 00004: val_loss improved from 4.09553 to 3.7

182/182 [==============================] - 136s 748ms/step - loss: 1.2060 - acc: 0.6866 - val_loss: 2.5308 - val_acc: 0.4212

Epoch 00035: val_loss did not improve from 2.51965
Epoch 36/300
182/182 [==============================] - 136s 746ms/step - loss: 1.1745 - acc: 0.7000 - val_loss: 2.5892 - val_acc: 0.4191

Epoch 00036: val_loss did not improve from 2.51965
Epoch 37/300
182/182 [==============================] - 136s 746ms/step - loss: 1.1778 - acc: 0.6867 - val_loss: 2.5423 - val_acc: 0.4238

Epoch 00037: val_loss did not improve from 2.51965
Epoch 38/300
182/182 [==============================] - 136s 745ms/step - loss: 1.1532 - acc: 0.7009 - val_loss: 2.5662 - val_acc: 0.4225

Epoch 00038: val_loss did not improve from 2.51965
Epoch 39/300
182/182 [==============================] - 135s 744ms/step - loss: 1.1793 - acc: 0.6935 - val_loss: 2.5679 - val_acc: 0.4233

Epoch 00039: val_loss did not improve from 2.51965
Epoch 40/300
182/182 [==============================] - 135s 74

KeyboardInterrupt: 

In [6]:

vals = pd.DataFrame.from_dict(Saving.history)
vals = pd.concat([pd.Series(range(0,100),name='epochs'),vals],axis=1)
vals.head(n=31)

NameError: name 'Saving' is not defined

In [ ]:
import seaborn as sns
sns.set_style({'xtick.bottom':False,
               'ytick.left':False,
               'axes.spines.bottom': False,
               'axes.spines.left': False,
               'axes.spines.right': False,
               'axes.spines.top': False})

ig,(ax,ax1) = plt.subplots(nrows=2,ncols=1,figsize=(20,20))
sns.scatterplot(x='epochs',y='acc',data=vals,ax=ax,color='r')
sns.lineplot(x='epochs',y='val_acc',data=vals,ax=ax,color='g')
sns.scatterplot(x='epochs',y='loss',data=vals,ax=ax1,color='r')
sns.lineplot(x='epochs',y='val_loss',data=vals,ax=ax1,color='g')
ax.legend(labels=['Test Accuracy','Training Accuracy'])
ax1.legend(labels=['Test Loss','Training Loss'])

In [ ]:
# for i in range(n_iterations):
#         print('\r{}/{}'. format(i, n_iterations))
#         #initilaize random hyperparameters
#         random_parameters = {K: random.sample(V, 1)[0] for K, V in parameters_grid.items()}
#         #create model with random hyperparameters
#         model = create_model(random_parameters)
#         opt = create_RMSprop_optimizer(random_parameters)
#         #compile model
#         model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])
#         #train model
#         epochs = 40
#         checkpointer = ModelCheckpoint(filepath='saved_models/model_weights.hdf5', verbose=1, save_best_only=True)
#         early_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 8)
#         history = model.fit(X_train, Y_train,
#                   callbacks = [checkpointer, early_stopping],
#                   validation_split = 0.33,
#                   epochs=epochs, batch_size = 20, verbose=1)
#         #compare current min validation loss with global min validation loss
#         current_min_val_loss = np.min(history.history['val_loss'])
#         if(current_min_val_loss < global_min_val_loss):
#             global_min_val_loss = current_min_val_loss
#             #save the best model so far
#             model.load_weights('saved_models/model_weights.hdf5')
#             model.save('saved_models/best_model.h5')
#             print('saving new best model, val_loss = {}'.format(current_min_val_loss))
#             #save best hyperparameters
#             best_hyperparameters = random_parameters